In [1]:
!pip install flask-ngrok
!pip install pyngrok
!pip install flask

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 681.2/681.2 kB 7.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyngrok: filename=pyngrok-6.0.0-py3-none-any.whl size=19867 sha256=faf501e1ba558af72d9a7e62617a4d13138fc6b11fd50177d5cff2a1d064aae4
  Stored in directory: /root/.cache/pip/wheels/5c/42/78/0c3d438d7f5730451a25f7ac6cbf4391759d22a67576ed7c2c
Successfully built pyngrok


In [2]:
from flask import Flask, request, jsonify
from pyngrok import ngrok
from flask_ngrok import run_with_ngrok

In [3]:
!ngrok config add-authtoken 2KkwwBb8LMpDEq5IpNsi7VF5O3T_4SJUvRLLnypoYNcsLxMYS

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


### Loading the model.
For now the process is:
 - Upload the model you want to use to the current environment
 - Run the function that recieves the raw data
 - Return the predictions

In [4]:
from tensorflow.keras.models import load_model
from sklearn.preprocessing import MinMaxScaler
import numpy as np
from flask import Flask, request, jsonify


In [5]:
def is_float(num):
    try:
        float(num)
        return True
    except ValueError:
        return False

In [6]:
def preprocess_sequences(sequences):
    sequences = np.array(sequences)
    # Shape: (SeuenceSize, 51)
    scaler = MinMaxScaler()
    normalized_sequences = np.zeros_like(sequences)
    for i in range(sequences.shape[0]):
        for j in range(sequences.shape[1]):
            # Flatten the landmarks for each set within the sequence
            landmarks_flattened = np.reshape(sequences[i, j], (-1, 1))
            # Normalize tshe landmarks
            landmarks_normalized = scaler.fit_transform(landmarks_flattened)
            # Reshape the normalized landmarks back to the original shape
            normalized_landmarks = np.reshape(landmarks_normalized, sequences[i, j].shape)
            # Update the normalized landmarks in the sequences array
            normalized_sequences[i, j] = normalized_landmarks
    return normalized_sequences

In [9]:
from flask import Flask
from flask_ngrok import run_with_ngrok

app = Flask(__name__)
run_with_ngrok(app)

# Load the .h5 model
model_path = 'lstm_model_ex1.h5'
model = load_model(model_path)
print(model.summary())
@app.route('/predict', methods=['POST'])
def home():
    try:
        data = request.json
        received_array = np.array(data['array'])
        processed_array = preprocess_sequences(received_array)
        # Adding one dimension since is only one sequence
        single_sequence = np.expand_dims(processed_array, axis=0)
        # this should return (1, 1) shape
        sequence_prediction = model.predict(single_sequence)
        response = {'response_array': sequence_prediction.tolist()}
        print(sequence_prediction)
        return jsonify(response), 200
    except Exception as e:
        return jsonify({'error': str(e)}), 400

app.run()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_2 (LSTM)               (None, 10, 128)           91648     
                                                                 
 dropout_2 (Dropout)         (None, 10, 128)           0         
                                                                 
 lstm_3 (LSTM)               (None, 64)                49408     
                                                                 
 dropout_3 (Dropout)         (None, 64)                0         
                                                                 
 dense_1 (Dense)             (None, 1)                 65        
                                                                 
Total params: 141,121
Trainable params: 141,121
Non-trainable params: 0
_________________________________________________________________
None
 * Serving Flask app '__main__'
 * Debug mo

INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


 * Running on http://8017-35-245-244-77.ngrok-free.app
 * Traffic stats available on http://127.0.0.1:4040


INFO:werkzeug:127.0.0.1 - - [31/Aug/2023 16:13:07] "GET / HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [31/Aug/2023 16:13:07] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [31/Aug/2023 16:13:39] "GET /predict HTTP/1.1" 405 -
